# Продвинутый pandas 2

**Задание 1** Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [28]:
import pandas as pd
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


In [29]:
active_users = ratings.groupby('userId').agg({'timestamp': ['min', 'max', ], 'rating': 'size'})
active_users = active_users[active_users['rating']['size'] > 100]
active_users.loc[:, 'start_date'] = pd.to_datetime(active_users['timestamp']['min'], unit='s')
active_users

timestamp             rating          start_date
               min         max   size                    
userId                                                   
4        949778714   949982274    204 2000-02-05 19:25:14
8       1154389340  1154474527    116 2006-07-31 23:42:20
15       997937239  1469330735   1700 2001-08-16 04:47:19
17      1127468587  1127476640    363 2005-09-23 09:43:07
19       855190091   855195373    423 1997-02-06 00:48:11
...            ...         ...    ...                 ...
656      986240991   986244044    128 2001-04-02 19:49:51
659      834598040   866207451    142 1996-06-12 16:47:20
664     1343731283  1441911722    519 2012-07-31 10:41:23
665      992836298  1046967769    434 2001-06-18 03:51:38
671     1063500751  1074784735    115 2003-09-14 00:52:31

[258 rows x 4 columns]

In [30]:
active_users.loc[:, 'end_date'] = pd.to_datetime(active_users['timestamp']['max'], unit='s')
active_users.loc[:,'ltv'] = active_users['end_date'] - active_users['start_date']
active_users['ltv'].mean()

Timedelta('463 days 21:28:27.449612')

**Задание 2** Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:

-таблицу с тремя типами выручки для каждого client_id без указания адреса клиента

-аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [24]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [25]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [26]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [27]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [11]:
rzd.merge(auto, on='client_id', how='outer').merge(air, on='client_id', how='outer')

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [12]:
rzd.merge(auto, on='client_id', how='outer').merge(air, on='client_id', how='outer').merge(client_base, on='client_id', how='outer')

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4


**Задание 3** В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:

У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить? Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1? Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.


In [35]:
#Я бы создала файл данных, в котором была бы представлена таблица примерно такого формата 

user_base = pd.DataFrame(
    {
        'user_id': [1, 2, 3, 4, 5, 6, 7, 8, 9],
        'visit_id': [1, 2, 1, 1, 2, 3, 7, 1, 8],
        'buy': [1, 1, 0, 0, 1, 1, 0, 0, 1],
        'latitude': [59.7, 59.3, 59.8, 59.7, 59.6, 59.9, 59.1, 59.2, 59.4],
        'longitude': [30.8, 30.2, 30.3, 30.1, 30.8, 30.5, 30.4, 30.9, 31.8],
    }
)
user_base

,user_id,visit_id,buy,latitude,longitude
0,1,1,1,59.7,30.8
1,2,2,1,59.3,30.2
2,3,1,0,59.8,30.3
3,4,1,0,59.7,30.1
4,5,2,1,59.6,30.8
5,6,3,1,59.9,30.5
6,7,7,0,59.1,30.4
7,8,1,0,59.2,30.9
8,9,8,1,59.4,31.8


Какие дополнительные признаки можно получить из координат?
наиболее популярные маршруты и точки координат, места где сделано больше всего покупок, средние пройденные расстояния, часовой пояс пользователя.